# Baseline with mse loss with wide cnn modified + adam
#### Convert histone mark signals and use deep CNN for training.

#### NOTE: Need to activate genomelake environment before this code. Simply type 'genomelake' in terminal.

In [1]:
%env CUDA_VISIBLE_DEVICES=4,5
import os, sys
sys.path.append("..")
import random
# custom file path package
from data import Data_Directories
# custom utility package
from utils.compute_util import *
# package for genomic data
from pybedtools import Interval, BedTool
from genomelake.extractors import ArrayExtractor, BigwigExtractor
# package for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats.stats import pearsonr,spearmanr
import tensorflow as tf
from sklearn.preprocessing import quantile_transform

env: CUDA_VISIBLE_DEVICES=4,5


Using TensorFlow backend.


In [2]:
window_size = 2001
sample_num = 10000

In [3]:
# retrieve data
data = Data_Directories()
print data.intervals.keys()
print data.input_atac['day0'].keys()
print data.output_histone['day0'].keys()

['day6', 'day3', 'day0']
['100', '140']
['H3K27me3', 'H3K4me1', 'H3K27ac']


In [4]:
# get intervals for day0 data
day0_intervals = list(BedTool(data.intervals['day0']))
print '# of Intervals Extracted for day0: {}'.format(len(day0_intervals))

# of Intervals Extracted for day0: 267226


In [5]:
# create an ArrayExtractor for ATAC-seq for day0 with 140 base pairs
bw_140bp_day0 = ArrayExtractor(data.input_atac['day0']['140'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [6]:
# create a BigWigExtractor for histone makr 'H3K27ac' for day0
bw_histone_mark_day0 = BigwigExtractor(data.output_histone['day0']['H3K27ac'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [7]:
# normalize day0 intervals
normalized_day0_intervals = [normalize_interval(interval, window_size) for interval in day0_intervals if normalize_interval(interval, window_size)]
print 'Finished normalizing day0 intervals!'

Finished normalizing day0 intervals!


In [8]:
assert (len(day0_intervals)==len(normalized_day0_intervals))
print "Examples of original intervals"
print [(int(_interval.start)+int(_interval[-1]), [int(_interval.start), int(_interval.end)])
       for _interval in day0_intervals[:3]]
print "Examples of normalized intervals with window size of {}".format(window_size)
print [([int(_interval.start), int(_interval.end)])
       for _interval in  normalized_day0_intervals[:3]]

Examples of original intervals
[(123412027, [123411855, 123412989]), (123411941, [123411855, 123412989]), (131908564, [131908487, 131910071])]
Examples of normalized intervals with window size of 2001
[[123411027, 123413028], [123410941, 123412942], [131907564, 131909565]]


In [9]:
atac_seq_day0 = bw_140bp_day0(normalized_day0_intervals)
print atac_seq_day0.shape

(267226, 2001, 5)


In [10]:
#TODO: put this into utils if possible
def prune_invalid_intervals(intervals, bigwig_file):
    for _interval in intervals[:]:
        try:
            bigwig_file([_interval])
        except:
            intervals.remove(_interval)
            pass
        
print "Before pruning day0: {}".format(len(normalized_day0_intervals))
prune_invalid_intervals(normalized_day0_intervals, bw_140bp_day0)
print "After pruning day0: {}".format(len(normalized_day0_intervals))

Before pruning day0: 267226
After pruning day0: 267226


In [11]:
print "Dimension of ATAC-seq signal: {}".format(bw_140bp_day0(normalized_day0_intervals[:1]).shape)

Dimension of ATAC-seq signal: (1, 2001, 5)


In [12]:
print "Dimension of histone mark signal: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1]).shape)

Dimension of histone mark signal: (1, 2001)


In [13]:
# replace nan values with zeros and convert it to p-values
histone_mark_day0 = np.nan_to_num(bw_histone_mark_day0(normalized_day0_intervals))
print histone_mark_day0.shape

(267226, 2001)


In [14]:
histone_mark_day0 = np.expand_dims(histone_mark_day0, axis=2)
print histone_mark_day0.shape

(267226, 2001, 1)


In [15]:
print "Example histone mark signal"
print "\tRaw value: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1])[0][:5].reshape(-1))

Example histone mark signal
	Raw value: [ 0.01014  0.01014  0.01014  0.02435  0.02435]


In [16]:
from keras.layers import Input, Dense, Conv1D, Dropout, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras import optimizers
from keras import metrics
from keras import losses
from keras import backend as K
from keras.callbacks import Callback, TensorBoard, ReduceLROnPlateau, ModelCheckpoint

In [17]:
dropout_rate = 0.5
# parameters for first conv layer
hidden_filters_1 = 32
hidden_kernel_size_1 = window_size
# parameters for second conv layer
output_filters = 1
output_kernel_size = 32
# parameters for training
batch_size = 128
num_epochs = 150
evaluation_freq = 10

In [18]:
'''
Baseline CNN (based on KERAS functional API)
'''
inputs = Input(shape=(window_size, 5, ))
x = Conv1D(
    filters=hidden_filters_1,
    kernel_size=hidden_kernel_size_1,
    padding='same',
    activation='relu',
    strides=1,
    name='gen_conv1d_1')(inputs)
x = Dropout(dropout_rate,
            name='gen_dropout_1')(x)
# x = BatchNormalization(momentum=0.8)(x)

outputs = Conv1D(
    filters=output_filters,
    kernel_size=output_kernel_size,
    padding='same',
    activation='linear',
    strides=1,
    name='gen_conv1d_output'
    )(x)

model = Model(inputs=inputs, outputs=outputs)

In [19]:
# Print out model summary
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2001, 5)           0         
_________________________________________________________________
gen_conv1d_1 (Conv1D)        (None, 2001, 32)          320192    
_________________________________________________________________
gen_dropout_1 (Dropout)      (None, 2001, 32)          0         
_________________________________________________________________
gen_conv1d_output (Conv1D)   (None, 2001, 1)           1025      
Total params: 321,217
Trainable params: 321,217
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
def pearson(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(np.multiply(xm,ym))
    r_den = K.sqrt(np.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den
    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return K.square(r)

In [21]:
# Helper functions for writing the scores into bigwig file
from itertools import izip
from itertools import groupby
import subprocess

def interval_key(interval):
    return (interval.chrom, interval.start, interval.stop)

def merged_scores(scores, intervals, merge_type):
    # A generator that returns merged intervals/scores
    # Scores should have shape: #examples x #categories x #interval_size
    # Second dimension can be omitted for a 1D signal
    signal_dims = scores.ndim - 1
    assert signal_dims in {1, 2}

    # Only support max for now
    assert merge_type == 'max'
    score_first_dim = 1 if signal_dims == 1 else scores.shape[1]

    dtype = scores.dtype

    sort_idx, sorted_intervals = \
        zip(*sorted(enumerate(intervals),
                    key=lambda item: interval_key(item[1])))
    sorted_intervals = BedTool(sorted_intervals)

    # Require at least 1bp overlap
    # Explicitly convert to list otherwise it will keep opening a file when
    # retrieving an index resulting in an error (too many open files)
    interval_clust = list(sorted_intervals.cluster(d=-1))
    for _, group in groupby(izip(sort_idx, interval_clust),
                            key=lambda item: item[1].fields[-1]):
        idx_interval_pairs = list(group)
        group_idx, group_intervals = zip(*idx_interval_pairs)

        if len(idx_interval_pairs) == 1:
            yield group_intervals[0], scores[group_idx[0], ...]
        else:
            group_chrom = group_intervals[0].chrom
            group_start = min(interval.start for interval in group_intervals)
            group_stop = max(interval.stop for interval in group_intervals)

            # This part needs to change to support more merge_types (e.g. mean)
            group_score = np.full((score_first_dim, group_stop - group_start),
                                  -np.inf, dtype)
            for idx, interval in idx_interval_pairs:
                slice_start = interval.start - group_start
                slice_stop = slice_start + (interval.stop - interval.start)
                group_score[..., slice_start:slice_stop] = \
                    np.maximum(group_score[..., slice_start:slice_stop],
                               scores[idx, ...])
            if signal_dims == 1:
                group_score = group_score.squeeze(axis=0)
            yield Interval(group_chrom, group_start, group_stop), group_score
            
def interval_score_pairs(intervals, scores, merge_type):
    return (izip(intervals, scores) if merge_type is None
            else merged_scores(scores, intervals, merge_type))

def _write_1D_deeplift_track(scores, intervals, file_prefix, merge_type='max',
                             CHROM_SIZES='/mnt/data/annotations/by_release/hg19.GRCh37/hg19.chrom.sizes'):
    assert scores.ndim == 2

    bedgraph = file_prefix + '.bedGraph'
    bigwig = file_prefix + '.bw'

    print 'Writing 1D track of shape: {}'.format(scores.shape)
    print 'Writing to file: {}'.format(bigwig)

    with open(bedgraph, 'w') as fp:
        for interval, score in interval_score_pairs(intervals, scores,
                                                    merge_type):
            chrom = interval.chrom
            start = interval.start
            for score_idx, val in enumerate(score):
                fp.write('%s\t%d\t%d\t%g\n' % (chrom,
                                               start + score_idx,
                                               start + score_idx + 1,
                                               val))
    print 'Wrote bedgraph.'

    try:
        output = subprocess.check_output(
            ['wigToBigWig', bedgraph, CHROM_SIZES, bigwig],
            stderr=subprocess.STDOUT)
        print 'wigToBigWig output: {}'.format(output)
    except subprocess.CalledProcessError as e:
        print 'wigToBigWig terminated with exit code {}'.format(
            e.returncode)
        print 'output was:\n' + e.output

    print 'Wrote bigwig.'

In [22]:
model_dir = os.path.join("models", "cnn_2000_norm_new")
log_dir = os.path.join("logs", "cnn_2000_norm_new")
srv_dir = os.path.join("/srv", "www", "kundaje", "jesikmin", "cnn_2000_norm_new")
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
if not os.path.exists(srv_dir):
    os.makedirs(srv_dir)

In [23]:
# setting an adam optimizer with 1.0 clip norm
adam = optimizers.Adam(lr=1e-3, clipnorm=1.)

print "Compiling a model with adam optimizer"
model.compile(loss=losses.mean_squared_error,
              optimizer=adam,
              metrics=[pearson, metrics.mse, metrics.mae])

Compiling a model with adam optimizer


In [24]:
# CallBack: tensorboard with grad histogram
tensorboard = TensorBoard(log_dir=log_dir,
                          histogram_freq=1,
                          batch_size=batch_size,
                          write_grads=True)

In [25]:
# CallBack: reduce learning rate when validation loss meets plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.96,
                              patience=5,
                              min_lr=1e-7)

In [26]:
# CallBack: evaluate model for every evaulation epoch
class EvaluateModel(Callback):
    def __init__(self, X_test, y_test):
        self.X_test, self.y_test = X_test, y_test
        self.epochs = 0

    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        # evaluate the model
        if self.epochs % evaluation_freq == 0:
            scores = model.evaluate(X_test, y_test)
            print zip(model.metrics_names, scores)
            test_prediction = model.predict(X_test)
            test_spearman_sum = 0.0
            for idx, _pred in enumerate(test_prediction):
                rho, _ = spearmanr(histone_mark_day0[10000+idx], _pred)
                test_spearman_sum += rho
            print "Test spearman: {}".format(test_spearman_sum * 1.0 / 1000)

In [27]:
# CallBack: store bigwig file for the best model
class SaveBigwig(Callback):
    def __init__(self, X_train, y_train, X_val, y_val, X_test, y_test):
        self.best_val_loss = float('Inf')
        self.best_epoch = -1
        self.X_train, self.y_train = X_train, y_train
        self.X_val, self.y_val = X_val, y_val
        self.X_test, self.y_test = X_test, y_test
        self.epochs = 0
    
    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        cur_val_loss = logs['val_loss']
        if cur_val_loss < self.best_val_loss:
            self.best_val_loss = cur_val_loss
            self.best_epoch = self.epochs
            prediction = model.predict(X_train).reshape(int(sample_num*0.8), window_size)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[:int(sample_num*0.8)], os.path.join(srv_dir, 'train'))
            prediction = model.predict(X_val).reshape(int(sample_num*0.2), window_size)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[int(sample_num*0.8):sample_num], os.path.join(srv_dir, 'val'))
            prediction = model.predict(X_test).reshape(int(sample_num*0.1), window_size)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[sample_num:int(sample_num*1.1)], os.path.join(srv_dir, 'test'))
            f = open(os.path.join(srv_dir, 'meta.txt'), 'wb')
            f.write(str(self.epochs) + "  " + str(logs))
            f.close()

In [28]:
# CallBack: Save model checkpoint based on validation loss
checkpointer = ModelCheckpoint(os.path.join(model_dir, "best_model.h5"),
                               monitor='val_loss',
                               verbose=1,
                               save_best_only=True,
                               mode='min')

In [54]:
print "Fitting the model..."
train_index = int(sample_num*0.8)
val_index = sample_num
test_index = int(sample_num*1.1) 

X_train, y_train = atac_seq_day0[:train_index], histone_mark_day0[:train_index]
X_train = np.log(X_train+1+10**-10)
X_train /= np.max(X_train)
X_train = (X_train - 0.5) * 2
print np.max(X_train), np.min(X_train)

X_val, y_val = atac_seq_day0[train_index:val_index], histone_mark_day0[train_index:val_index]
X_val = np.log(X_val+1+10**-10)
X_val /= np.max(X_val)
X_val = (X_val - 0.5) * 2
print np.max(X_val), np.min(X_val)

X_test, y_test = atac_seq_day0[val_index:test_index], histone_mark_day0[val_index:test_index]
x_test = np.log(X_test+1+10**-10)
X_test /= np.max(X_test)
X_test = (X_test - 0.5) * 2
print np.max(X_test), np.min(X_test)

model.fit(atac_seq_day0[:sample_num],
          histone_mark_day0[:sample_num],
          batch_size=batch_size,
          epochs=num_epochs,
          validation_split=0.2,   
          #shuffle=True,
          callbacks=[tensorboard,
                     reduce_lr,
                     EvaluateModel(X_test, y_test),
                     checkpointer,
                     SaveBigwig(X_train, y_train, X_val, y_val, X_test, y_test)])

Fitting the model...
1.0 -1.0
1.0 -1.0
1.0 -1.0
Train on 8000 samples, validate on 2000 samples
Epoch 1/150
7936/8000 [============================>.] - ETA: 0s - loss: 364.8990 - pearson: 0.1999 - mean_squared_error: 364.8990 - mean_absolute_error: 7.9596
Epoch 00001: val_loss improved from 132.13225 to 130.81945, saving model to models/cnn_2000_norm_new/best_model.h5
Writing 1D track of shape: (8000, 2001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_2000_norm_new/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2000, 2001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_2000_norm_new/val.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (1000, 2001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_2000_norm_new/test.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
8000/8000 [==============================] - 159s 20ms/step - loss: 364.7033 - pearson: 0.1997 - mean_squared_error: 364.7033 - mean_absolute_error:

Epoch 15/150
7936/8000 [============================>.] - ETA: 0s - loss: 356.3759 - pearson: 0.2202 - mean_squared_error: 356.3759 - mean_absolute_error: 7.9445
Epoch 00015: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 356.0722 - pearson: 0.2191 - mean_squared_error: 356.0722 - mean_absolute_error: 7.9432 - val_loss: 130.2129 - val_pearson: 0.1554 - val_mean_squared_error: 130.2129 - val_mean_absolute_error: 3.9725
Epoch 16/150
7936/8000 [============================>.] - ETA: 0s - loss: 355.8525 - pearson: 0.2190 - mean_squared_error: 355.8525 - mean_absolute_error: 7.8555
Epoch 00016: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 356.3076 - pearson: 0.2189 - mean_squared_error: 356.3076 - mean_absolute_error: 7.8647 - val_loss: 130.7053 - val_pearson: 0.1560 - val_mean_squared_error: 130.7053 - val_mean_absolute_error: 4.2929
Epoch 17/150
7936/8000 [============================>.] - ETA: 0s - 

Epoch 32/150
7936/8000 [============================>.] - ETA: 0s - loss: 353.4577 - pearson: 0.2274 - mean_squared_error: 353.4577 - mean_absolute_error: 7.8896
Epoch 00032: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 353.2769 - pearson: 0.2266 - mean_squared_error: 353.2769 - mean_absolute_error: 7.8945 - val_loss: 130.6638 - val_pearson: 0.1527 - val_mean_squared_error: 130.6638 - val_mean_absolute_error: 4.2368
Epoch 33/150
7936/8000 [============================>.] - ETA: 0s - loss: 354.9420 - pearson: 0.2257 - mean_squared_error: 354.9420 - mean_absolute_error: 7.8420
Epoch 00033: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 354.1823 - pearson: 0.2251 - mean_squared_error: 354.1823 - mean_absolute_error: 7.8314 - val_loss: 131.5477 - val_pearson: 0.1543 - val_mean_squared_error: 131.5477 - val_mean_absolute_error: 4.5692
Epoch 34/150
7936/8000 [============================>.] - ETA: 0s - 

7936/8000 [============================>.] - ETA: 0s - loss: 348.0553 - pearson: 0.2274 - mean_squared_error: 348.0553 - mean_absolute_error: 7.9239
Epoch 00049: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 351.3877 - pearson: 0.2275 - mean_squared_error: 351.3877 - mean_absolute_error: 7.9362 - val_loss: 130.5474 - val_pearson: 0.1531 - val_mean_squared_error: 130.5474 - val_mean_absolute_error: 4.2044
Epoch 50/150
1000/1000 [==============================] - 1s 654us/steps: 351.3388 - pearson: 0.2246 - mean_squared_error: 351.3388 - mean_
[('loss', 614369.58200000005), ('pearson', 0.0045323797013093097), ('mean_squared_error', 614369.58550000004), ('mean_absolute_error', 652.75798144531245)]
Test spearman: 0.0494135782135

Epoch 00050: val_loss did not improve
8000/8000 [==============================] - 18s 2ms/step - loss: 351.2996 - pearson: 0.2256 - mean_squared_error: 351.2996 - mean_absolute_error: 7.9363 - val_loss: 132.3486 - val_

Epoch 66/150
7936/8000 [============================>.] - ETA: 0s - loss: 349.5284 - pearson: 0.2357 - mean_squared_error: 349.5284 - mean_absolute_error: 7.9365
Epoch 00066: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 349.7207 - pearson: 0.2348 - mean_squared_error: 349.7207 - mean_absolute_error: 7.9343 - val_loss: 130.9339 - val_pearson: 0.1538 - val_mean_squared_error: 130.9339 - val_mean_absolute_error: 4.4117
Epoch 67/150
7936/8000 [============================>.] - ETA: 0s - loss: 347.4167 - pearson: 0.2301 - mean_squared_error: 347.4167 - mean_absolute_error: 8.0058
Epoch 00067: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 349.5525 - pearson: 0.2293 - mean_squared_error: 349.5525 - mean_absolute_error: 8.0082 - val_loss: 132.2387 - val_pearson: 0.1476 - val_mean_squared_error: 132.2387 - val_mean_absolute_error: 3.8969
Epoch 68/150
7936/8000 [============================>.] - ETA: 0s - 

Epoch 83/150
7936/8000 [============================>.] - ETA: 0s - loss: 349.3889 - pearson: 0.2344 - mean_squared_error: 349.3889 - mean_absolute_error: 8.1177
Epoch 00083: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 347.9679 - pearson: 0.2339 - mean_squared_error: 347.9679 - mean_absolute_error: 8.1056 - val_loss: 131.3354 - val_pearson: 0.1515 - val_mean_squared_error: 131.3354 - val_mean_absolute_error: 4.3498
Epoch 84/150
7936/8000 [============================>.] - ETA: 0s - loss: 348.5574 - pearson: 0.2290 - mean_squared_error: 348.5574 - mean_absolute_error: 7.8398
Epoch 00084: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 348.7518 - pearson: 0.2316 - mean_squared_error: 348.7518 - mean_absolute_error: 7.8390 - val_loss: 131.2583 - val_pearson: 0.1515 - val_mean_squared_error: 131.2583 - val_mean_absolute_error: 4.2198
Epoch 85/150
7936/8000 [============================>.] - ETA: 0s - 

1000/1000 [==============================] - 1s 658us/steps: 346.6369 - pearson: 0.2372 - mean_squared_error: 346.6369 - mean_
[('loss', 9530915.1760000009), ('pearson', 0.0034292533533429718), ('mean_squared_error', 9530915.1999999993), ('mean_absolute_error', 2760.4871308593752)]
Test spearman: 0.0725865916126

Epoch 00100: val_loss did not improve
8000/8000 [==============================] - 18s 2ms/step - loss: 345.5029 - pearson: 0.2367 - mean_squared_error: 345.5029 - mean_absolute_error: 7.9447 - val_loss: 131.7260 - val_pearson: 0.1499 - val_mean_squared_error: 131.7260 - val_mean_absolute_error: 4.1598
Epoch 101/150
7936/8000 [============================>.] - ETA: 0s - loss: 348.3280 - pearson: 0.2349 - mean_squared_error: 348.3280 - mean_absolute_error: 8.0013
Epoch 00101: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 347.5764 - pearson: 0.2347 - mean_squared_error: 347.5764 - mean_absolute_error: 7.9980 - val_loss: 131.7070 - val

Epoch 117/150
7936/8000 [============================>.] - ETA: 0s - loss: 343.1721 - pearson: 0.2388 - mean_squared_error: 343.1721 - mean_absolute_error: 8.0472
Epoch 00117: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 346.0554 - pearson: 0.2399 - mean_squared_error: 346.0554 - mean_absolute_error: 8.0715 - val_loss: 131.1967 - val_pearson: 0.1521 - val_mean_squared_error: 131.1967 - val_mean_absolute_error: 4.3156
Epoch 118/150
7936/8000 [============================>.] - ETA: 0s - loss: 346.5729 - pearson: 0.2422 - mean_squared_error: 346.5729 - mean_absolute_error: 7.9860
Epoch 00118: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 345.3471 - pearson: 0.2413 - mean_squared_error: 345.3471 - mean_absolute_error: 7.9804 - val_loss: 130.8397 - val_pearson: 0.1520 - val_mean_squared_error: 130.8397 - val_mean_absolute_error: 4.3563
Epoch 119/150
7936/8000 [============================>.] - ETA: 0s

Epoch 134/150
7936/8000 [============================>.] - ETA: 0s - loss: 346.9668 - pearson: 0.2400 - mean_squared_error: 346.9668 - mean_absolute_error: 8.0480
Epoch 00134: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 345.4695 - pearson: 0.2396 - mean_squared_error: 345.4695 - mean_absolute_error: 8.0330 - val_loss: 131.0776 - val_pearson: 0.1501 - val_mean_squared_error: 131.0776 - val_mean_absolute_error: 4.1673
Epoch 135/150
7936/8000 [============================>.] - ETA: 0s - loss: 348.2366 - pearson: 0.2393 - mean_squared_error: 348.2366 - mean_absolute_error: 7.9884
Epoch 00135: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 346.8338 - pearson: 0.2404 - mean_squared_error: 346.8338 - mean_absolute_error: 7.9814 - val_loss: 133.1031 - val_pearson: 0.1535 - val_mean_squared_error: 133.1031 - val_mean_absolute_error: 4.7093
Epoch 136/150
7936/8000 [============================>.] - ETA: 0s